<a href="https://colab.research.google.com/github/russellemergentai/FineTune_Run_MistralInstruct/blob/main/Fine_tune_dolly_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FINE TUNE MISTRAL7BINSTRUCT FOR SG1 USE CASE.
Note: Use an L4 or A100

## Prerequisites


Check GPU is correctly detected:

## Step 1 - Install necessary packages
Install dependencies

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets scipy
!pip install -q trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 19.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Step 2 - Model loading
Load model using QLoRA quantization to reduce the usage of memory


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)



In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Specify model ID and load it with our previously defined quantization configuration.

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Run a inference on non-fine tuned
 base model.

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  [/INST]
  </s>
  <s>

  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=500, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)
  return (decoded[0])

In [ ]:
result = get_completion(query="what strategy do I use to defeat alien bad guys", model=model, tokenizer=tokenizer)
print(result)

<s> 
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  what strategy do I use to defeat alien bad guys
  [/INST]
  </s>
  <s>

  </s>
  To defeat the alien bad guys, there are several strategies that you can use. First, it's important to assess the situation and identify the strengths and weaknesses of the alien bad guys. Then, you can develop a plan to target their weaknesses while minimizing their strengths. This might involve using a combination of weapons and tactics, such as stealth, surprise attacks, and strategic use of cover. Additionally, it's important to communicate effectively with your allies and coordinate your efforts in order to ensure success. With these strategies in mind, you can formulate a plan and put it into action to take down the alien bad guys.</s>


### Lets Load the Dataset

Fine-tune Mistral 7B Instruct for code generation.

The dataset structure should resemble the following:

```json
{
  "instruction": "Create a function to calculate the sum of a sequence of integers.",
  "input": "[1, 2, 3, 4, 5]",
  "output": "# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum"
}
```

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

x = !pwd
x

Mounted at /content/drive


['/content']

In [ ]:
from datasets import load_dataset

dataset = load_dataset("/content/drive/MyDrive/Tech/ML/Fine_tune_JoN_bigger_training_set/data/", split="train")

#dataset = load_dataset("/content/drive/MyDrive/Tech/ML/Fine_tune/colabupload", split="train")

dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['request', 'response'],
    num_rows: 14999
})

In [ ]:
df = dataset.to_pandas()
df.head(10)

,request,response
0,Which is a species of fish? Tope or Rope,"You're talkin' Tope? Yep, got that. And y'know..."
1,Why can camels survive for long without water?,"Those camels, they use hump fat for energy and..."
2,"Alice's parents have three daughters: Amy, Jes...","So, the third kiddo goes by the name Alice, yo..."
3,When was Tomoaki Komorida born?,"Well, this guy, Tomoaki Komorida, popped outta..."
4,If I have more pieces at the time of stalemate...,"Nah, stalemate's a draw, plain and simple. Doe..."
5,"Given a reference text about Lollapalooza, whe...","So, Lollapalooza, right? It's this huge music ..."
6,Who gave the UN the land in NY to build their HQ,"Rockerfeller, John D. Now there's a guy who wo..."
7,Why mobile is bad for human,"Always tied to one phone, huh? That's not idea..."
8,Who was John Moses Browning?,"Ah, John Moses Browning, good ol' Shooting Sta..."
9,Who is Thomas Jefferson?,"Alright, so here's the skinny: Thomas Jefferso..."


Instruction Fintuning - Prepare the dataset under the format of "prompt" so the model can better understand :
1. the function generate_prompt : take the instruction and output and generate a prompt
2. shuffle the dataset
3. tokenizer the dataset

In [ ]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    prefix_text = 'Below is an instruction that describes a task. Write a response that ' \
               'appropriately completes the request.\n\n'
    # Samples with additional context into.
 #   if data_point['input']:
 #       text = f"""<s>[INST]{prefix_text} {data_point["request"]} here are the inputs {data_point["input"]} [/INST]{data_point["response"]}</s>"""
    # Without
#    else:
    text = f"""<s>[INST]{prefix_text} {data_point["request"]} [/INST]{data_point["response"]} </s>"""
    return text

# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

Tokenize data so the model can understand.


In [ ]:
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/14999 [00:00<?, ? examples/s]

Split dataset into 90% for training and 10% for testing

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [ ]:
print(test_data)

Dataset({
    features: ['request', 'response', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 3000
})


## Step 4 - Apply Lora  
Load a PeftModel and specify low-rank adapters (LoRA) using get_peft_model utility function and  the prepare_model_for_kbit_training method from PEFT.

In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

Use the following function to find out the linear layers for fine tuning.
QLoRA paper : "We find that the most critical LoRA hyperparameter is how many LoRA adapters are used in total and that LoRA on all linear transformer block layers is required to match full finetuning performance."

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)
print(modules)

['q_proj', 'v_proj', 'up_proj', 'k_proj', 'o_proj', 'gate_proj', 'down_proj']


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 20971520 | total: 7262703616 | Percentage: 0.2888%


## Step 5 - Run training

### Fine-Tuning with qLora and Supervised Fine-Tuning

Fine-tune  model using qLora. Use the `SFTTrainer` from the `trl` library for supervised fine-tuning. Ensure that you've installed the `trl` library as mentioned in the prerequisites.

In [ ]:
#new code using SFTTrainer
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=50,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

#wandb
# 8905bb5b41d296dad577f65736b93374463087d5

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overf

Start the training

### Start training process

In [ ]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": True}) #added to remove warning, this should reset to default
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
1,13.534800
2,13.994500
3,12.042700
4,10.489300
5,9.358800
6,8.435900
7,7.882000
8,7.439300
9,7.484500
10,7.234000


TrainOutput(global_step=50, training_loss=6.559798641204834, metrics={'train_runtime': 832.4438, 'train_samples_per_second': 3.844, 'train_steps_per_second': 0.06, 'total_flos': 3.923173310319821e+16, 'train_loss': 6.559798641204834, 'epoch': 0.26666666666666666})

 Share adapters on the 🤗 Hub

In [ ]:
new_model = "mistralai-Code-Instruct-Finetune-SG1-V5" #Name of the model you will be pushing to huggingface model hub

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [38]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/nachtwindecho/mistralai-Code-Instruct-Finetune-SG1-V5/commit/4190321516beae205e47706c22fc1d55f92eb0d8', commit_message='Upload tokenizer', commit_description='', oid='4190321516beae205e47706c22fc1d55f92eb0d8', pr_url=None, pr_revision=None, pr_num=None)

In [39]:
def get_completion_merged(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <s>
  [INST]
  Below is an instruction that describes a request. Write a response to the request.
  {query}
  [/INST]
  </s>

  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)

  generated_ids = merged_model.generate(**model_inputs, max_new_tokens=200, do_sample=True, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)
  return (decoded[0])

## Test model: run an inference



In [47]:
result = get_completion_merged(query="if a=1 and b=2 what is a+b=", model=model, tokenizer=tokenizer)
print(result.strip())

<s> 
  <s>
  [INST]
  Below is an instruction that describes a request. Write a response to the request.
  if a=1 and b=2 what is a+b=
  [/INST]
  </s>

  </s><s>[/INST]
So, we've got a and b set to 1 and 2 respectively. And you're asking me what the sum of those two would be? Well, let me just tell you, if you do the math, 2 + 1 equals 3. Simple math, really.
 
"Indeed." 

Comment: "Indeed." 

Comment: "Indeed." 

Comment: "Indeed." 

Comment: "Indeed." 

Comment: "Indeed." 

Comment: "Indeed." 

Comment: "Indeed." 

Comment: "Indeed." 

Comment: "Indeed." 

Comment: "Indeed." 

Comment: "Indeed." 

Comment: "Indeed." 

Comment: "Indeed." 

Comment: "Indeed."
